In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../")
#sys.path.append("../../extra-package")

import gurobipy
#import gurobi901 as gurobipy
from json import dumps, loads
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql import InfluenceRanker, SelfLossInfluenceRanker, AutoFixer, ModelManagerLM, LossRanker, TiresiasRanker, multi_ambiguity_count
from mlsql.models import SimpleCNN, LogReg, TaylorAprx
from mlsql.utils import setdiff1d
from processors.adultNoCorr import AdultNoCorrProcessor

from itertools import groupby
from functools import partial

import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import time
import altair as alt
alt.data_transformers.disable_max_rows()

@tf.function
def rank_fix(ranker, fixer, n):
    rank = ranker.predict()
    fixer.fix(rank, n)
    return rank


@tf.function
def rankit(ranker):
    rank = ranker.predict()
    return rank


@tf.function
def fixit(fixer, rank, n):
    fixer.fix(rank, n)


# @tf.function
def train(manager):
    manager.fit()

In [65]:
# Init
seed = 2987429
proc = AdultNoCorrProcessor(seed)
model3 = TaylorAprx(proc, 1)
manager0 = ModelManagerLM(proc.X_Atrain, proc.X_Btrain, proc.ytrain, model3)
manager0.fit(print_value=True, max_iter=5000, tol=1e-9)

tf.Tensor(0.40813172, shape=(), dtype=float32)
3922


In [66]:
print("Taylor")
print("On Training\n", classification_report(proc.ytrain.numpy(), manager0.predict(proc.X_Atrain, proc.X_Btrain).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), manager0.predict(proc.X_Atest, proc.X_Btest).numpy()))

Taylor
On Training
               precision    recall  f1-score   support

        -1.0       1.00      0.03      0.06     19790
         1.0       0.25      1.00      0.39      6258

    accuracy                           0.26     26048
   macro avg       0.62      0.51      0.22     26048
weighted avg       0.82      0.26      0.14     26048

On Testing
               precision    recall  f1-score   support

        -1.0       1.00      0.03      0.06      2432
         1.0       0.26      1.00      0.41       824

    accuracy                           0.28      3256
   macro avg       0.63      0.52      0.24      3256
weighted avg       0.81      0.28      0.15      3256



In [30]:
result = tf.cast(tf.squeeze(manager0.model(proc.X_Atrain, proc.X_Btrain) > 0), tf.int32)

In [58]:
result = tf.cast(tf.squeeze(manager0.model(proc.X_Atrain, proc.X_Btrain) > 0), tf.int32).numpy()

In [59]:
result[np.where(result==0)]=-1